In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import joblib
import optuna
from scipy.stats import rankdata
import warnings
warnings.filterwarnings("ignore") 

C:\Users\Eugene\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Считываем блендинг и параметры для моделей

In [4]:
from new_params_boost import *

from blender import *

In [5]:
lgb_par = lgbm_par
hist_par = hist_par
for_par = for_par
lgbm_dart_par = lgbm_dart_par
cat_categ1_par = cat_ordered_symtree_par
cat_categ2_par = cat_plain_symtree_par
cat_categ3_par = cat_plain_depthwise_par
cat_categ4_par = cat_plain_lossguide_par
cat_reg1_par = cat_reg_ordered_symtree_par
cat_reg2_par = cat_reg_plain_symtree_par
cat_reg3_par = cat_reg_plain_depthwise_par
cat_reg4_par = cat_reg_plain_lossguide_par
xgb_params = xgb_params

In [6]:
def get_rank_by_preds(predict):
    return rankdata(predict)

In [7]:
def model_fit(X_train, y_train, models):
    fitted_models = []
    
    for model in models:
        fitted_models.append(model.fit(X_train, y_train))

    return fitted_models

In [8]:
def get_preds(X, models_cls, models_reg=[], rank=True):
    preds = []
    
    for model in models_cls:
        if rank:
            preds.append(get_rank_by_preds(model.predict_proba(X)[:, 1]))
        else:
            preds.append(model.predict_proba(X)[:, 1])


    for model in models_reg:
        preds.append(get_rank_by_preds(model.predict(X)))

    return preds

# Считываем предпоготовленные данные и оставляем отобранные фичи

In [9]:
train = pd.read_csv('train_v7.csv')
oot = pd.read_csv('oot_v6.csv')
oot = train[train.id.isin(oot.id)]
train = train[~train.id.isin(oot.id)]

In [10]:
train.shape[0] + oot.shape[0]

38731

In [11]:
features = [i for i in train.columns.tolist() if not(i in ['id', 'target', 'smpl'])]

In [12]:
features = ['feature_65', 'feature_74', 'feature_51', 'feature_290', 'feature_167',
       'feature_328', 'feature_358', 'feature_37', 'feature_278',
       'feature_391', 'feature_52', 'feature_104', 'feature_6', 'feature_118',
       'feature_158', 'feature_276', 'feature_277', 'feature_263',
       'feature_82', 'feature_83', 'feature_378', 'feature_360', 'feature_7',
       'feature_354', 'feature_11', 'feature_366', 'feature_327', 'feature_61',
       'feature_269', 'feature_321', 'feature_256', 'feature_86',
       'feature_414', 'feature_33', 'feature_300', 'feature_58', 'feature_418',
       'feature_94', 'feature_67', 'feature_191', 'feature_346', 'feature_386',
       'feature_199', 'feature_117', 'feature_309', 'feature_16',
       'feature_215', 'feature_19', 'feature_63', 'feature_324', 'feature_100',
       'feature_174', 'feature_45', 'feature_296', 
       'mean_by_row']

In [13]:
X_train = train.loc[:, features]
y_train = train.loc[:, 'target']

In [14]:
X_oot = oot[features]
y_oot = oot.target

# Прописываем модели: отдельно классификации и регрессии

In [15]:
models_cls_cl = [
    CatBoostClassifier(**cat_categ1_par, verbose=0, boosting_type='Ordered', grow_policy='SymmetricTree'),
    CatBoostClassifier(**cat_categ2_par, verbose=0, boosting_type='Plain', grow_policy='SymmetricTree'),
    CatBoostClassifier(**cat_categ3_par, verbose=0, boosting_type='Plain', grow_policy='Depthwise'),
    CatBoostClassifier(**cat_categ4_par, verbose=0, boosting_type='Plain', grow_policy='Lossguide'),
    LGBMClassifier(**lgb_par, verbose=-1),
    LGBMClassifier(**lgbm_dart_par, verbose=-1, boosting_type='dart'),
    HistGradientBoostingClassifier(**hist_par),
    RandomForestClassifier(**for_par),
    XGBClassifier(**xgb_params)
]

In [16]:
models_reg_cl = [
    CatBoostRegressor(**cat_reg1_par, verbose=0, boosting_type='Ordered', grow_policy='SymmetricTree'),
    CatBoostRegressor(**cat_reg2_par, verbose=0, boosting_type='Plain', grow_policy='SymmetricTree'),
    CatBoostRegressor(**cat_reg3_par, verbose=0, boosting_type='Plain', grow_policy='Depthwise'),
    CatBoostRegressor(**cat_reg4_par, verbose=0, boosting_type='Plain', grow_policy='Lossguide')
]

# Обучаем модели на обучающей выборке b g

In [17]:
cls_models = model_fit(X_train, y_train, models_cls_cl)

In [18]:
reg_models = model_fit(X_train, y_train, models_reg_cl)

In [19]:
preds = get_preds(X_train, cls_models, reg_models, rank=True)

fin_preds = pd.DataFrame(preds).T.values

In [20]:
oot_preds = get_preds(X_oot, cls_models, reg_models, rank=True)

fin_oot_preds = pd.DataFrame(oot_preds).T.values

In [21]:
for i in range(fin_preds.shape[1]):
    print(roc_auc_score(y_train, fin_preds[:, i]))

0.8686227632401293
0.9272529162824918
0.7493736731170555
0.9162873489074452
0.9489344996991496
0.9807501472140124
0.9399382082479184
0.8973436450711304
0.9314727903949429
0.9505854642068245
0.9355922739838421
0.9569554255638988
0.957446098798565


In [22]:
for i in range(fin_oot_preds.shape[1]):
    print(roc_auc_score(y_oot, fin_oot_preds[:, i]))

0.7893165299193163
0.786290597727492
0.7459346389234646
0.7666243914605652
0.8124204398660684
0.811453365114249
0.787201200960957
0.8048061968079475
0.7965354723489563
0.7910083224900175
0.7914106820582928
0.763697284425861
0.7423745802872046


In [23]:
blend_results = []

for seed in range(5):
    blend_results.append(get_composition(seed, fin_preds.shape[1], y_oot, fin_oot_preds))

Started iteration 0
Update weight idx = 5 from 0.07692307692307693 to 0.86. Best score updates to 0.8119004313012215
New weights = [0.01166667 0.01166667 0.01166667 0.01166667 0.01166667 0.86
 0.01166667 0.01166667 0.01166667 0.01166667 0.01166667 0.01166667
 0.01166667], their sum = 1.0000000000000002
Update weight idx = 9 from 0.01166666666666667 to 0.29. Best score updates to 0.8151381316868749
New weights = [0.00838111 0.00838111 0.00838111 0.00838111 0.00838111 0.61780776
 0.00838111 0.00838111 0.00838111 0.29       0.00838111 0.00838111
 0.00838111], their sum = 0.9999999999999997
Update weight idx = 3 from 0.008381112984822935 to 0.0. Best score updates to 0.8153240171014582
New weights = [0.00845195 0.00845195 0.00845195 0.         0.00845195 0.62302944
 0.00845195 0.00845195 0.00845195 0.29245107 0.00845195 0.00845195
 0.00845195], their sum = 1.0
Update weight idx = 7 from 0.00845194973045593 to 0.05. Best score updates to 0.8153804886198125
New weights = [0.00809779 0.008097

In [24]:
best_blend_w = sorted(blend_results, reverse=True, key=lambda x: x[1])[0][2]

In [25]:
best_blend_w

array([0.        , 0.        , 0.        , 0.        , 0.21      ,
       0.55902182, 0.        , 0.0225037 , 0.00143436, 0.20704012,
       0.        , 0.        , 0.        ])

# Обучаем модель на обучающей и оот выборках

In [26]:
X_final = pd.concat([X_train, X_oot])
y_final = pd.concat([y_train, y_oot])

In [27]:
cls_models = model_fit(X_final, y_final, models_cls_cl)

In [28]:
reg_models = model_fit(X_final, y_final, models_reg_cl)

In [29]:
final_train_preds = get_preds(X_final, cls_models, reg_models)

In [30]:
roc_auc_score(y_final, calculate_prediction(final_train_preds, best_blend_w))

0.9625483537919591

# Считываем и делаем предсказания на тестовых данных

In [31]:
data_test = pd.read_csv('test_v7.csv')
data_test.shape

(19826, 204)

In [32]:
test_preds = get_preds(data_test[features], cls_models, reg_models)

In [33]:
y_test_pred = calculate_prediction(test_preds, best_blend_w)

In [34]:
data_test['target'] = y_test_pred

# Сохраним итоговые данные об id и предсказаниях в формате csv
data_test[['id', 'target']].to_csv('submit_evgen_15.csv', index=False)

In [ ]:
data_test[['id', 'target']].to_csv('../blending_submissions/ensemble_1.csv', index=False)